In [2]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain.chat_models import init_chat_model

In [3]:
class ModelResponse(BaseModel):
    """Model response schema."""
    model_name: str = Field(description="Name of the model", max_length=20)
    response: str = Field(description="Response from the model", max_length=100)
    tagline: str = Field(description="response created by Vivek", max_length=50)

In [4]:
resp_parser = PydanticOutputParser(pydantic_object=ModelResponse)

In [5]:
model = init_chat_model(model="gemini-2.0-flash", model_provider="google_genai")